In [13]:
"""
Advanced Tourism Recommendation System - Deep Learning Backend
Uses Pre-trained Embeddings + Hybrid Neural Architecture
Optimized for Google Colab with 5-6 Key Parameters
TRAINING MODE ONLY
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Deep Learning imports
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, Model
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.regularizers import l2
    print("✓ TensorFlow loaded successfully")
except:
    print("Installing TensorFlow...")
    import os
    os.system('pip install tensorflow -q')
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, Model
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.regularizers import l2

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


class AdvancedTourismRecommender:
    """
    Advanced Deep Learning Recommendation System

    Key Parameters (5-6):
    1. State - Location preference
    2. NumberOfAdults - Group size
    3. NumberOfChildren - Family composition
    4. Preferences - User interests (e.g., Adventure, Cultural, Beach)
    5. BestTimeToVisit - Seasonal preference
    6. Type - Destination type (e.g., Historical, Nature, Religious)

    Architecture:
    - Pre-trained embeddings for text (ReviewText, Preferences)
    - Neural Collaborative Filtering (NCF)
    - Multi-modal feature fusion
    - Attention mechanism for preference weighting
    """

    def __init__(self, csv_path):
        self.csv_path = csv_path
        self.df = None
        self.model = None

        # Encoders
        self.user_encoder = LabelEncoder()
        self.dest_encoder = LabelEncoder()
        self.state_encoder = LabelEncoder()
        self.type_encoder = LabelEncoder()
        self.time_encoder = LabelEncoder()
        self.pref_encoder = LabelEncoder()

        # Scalers
        self.feature_scaler = StandardScaler()
        self.rating_scaler = MinMaxScaler()

        # Storage
        self.destination_features = None
        self.user_profiles = {}
        self.similarity_matrix = None

    def load_and_analyze_data(self):
        """Load CSV and perform comprehensive analysis"""
        print("=" * 70)
        print("📊 LOADING & ANALYZING TOURISM DATASET")
        print("=" * 70)

        self.df = pd.read_csv(self.csv_path)

        print(f"\n✓ Loaded {len(self.df):,} records")
        print(f"✓ Columns: {len(self.df.columns)}")

        print(f"\n📋 Available Columns:")
        for i, col in enumerate(self.df.columns, 1):
            print(f"  {i:2d}. {col}")

        print(f"\n🔍 Data Quality Report:")
        print(f"  Total Records: {len(self.df):,}")
        print(f"  Missing Values:")
        missing = self.df.isnull().sum()
        for col in missing[missing > 0].index:
            pct = (missing[col] / len(self.df)) * 100
            print(f"    - {col}: {missing[col]:,} ({pct:.1f}%)")

        print(f"\n📈 Key Statistics:")
        if 'Rating' in self.df.columns:
            print(f"  Avg Rating: {self.df['Rating'].mean():.2f}")
            print(f"  Rating Range: {self.df['Rating'].min():.1f} - {self.df['Rating'].max():.1f}")

        if 'State' in self.df.columns:
            print(f"  Unique States: {self.df['State'].nunique()}")
            print(f"  Top 5 States: {', '.join(self.df['State'].value_counts().head(5).index.tolist())}")

        if 'Type' in self.df.columns:
            print(f"  Destination Types: {', '.join(self.df['Type'].unique()[:5].tolist())}")

        return self.df

    def preprocess_data(self):
        """Advanced preprocessing with feature engineering"""
        print("\n" + "=" * 70)
        print("🔧 ADVANCED PREPROCESSING & FEATURE ENGINEERING")
        print("=" * 70)

        df = self.df.copy()

        # Handle missing values intelligently
        df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(df['Rating'].median())
        df['ExperienceRating'] = pd.to_numeric(df['ExperienceRating'], errors='coerce').fillna(df['Rating'])
        df['NumberOfAdults'] = pd.to_numeric(df['NumberOfAdults'], errors='coerce').fillna(2)
        df['NumberOfChildren'] = pd.to_numeric(df['NumberOfChildren'], errors='coerce').fillna(0)
        df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce').fillna(df['Popularity'].median() if 'Popularity' in df.columns else 50)

        # Fill categorical nulls
        df['State'] = df['State'].fillna('Unknown')
        df['Type'] = df['Type'].fillna('General')
        df['BestTimeToVisit'] = df['BestTimeToVisit'].fillna('Year-round')
        df['Preferences'] = df['Preferences'].fillna('General Tourism')
        df['Gender'] = df['Gender'].fillna('Unknown')

        # Clean text fields
        if 'ReviewText' in df.columns:
            df['ReviewText'] = df['ReviewText'].fillna('').astype(str)
            df['review_length'] = df['ReviewText'].str.len()
            df['review_word_count'] = df['ReviewText'].str.split().str.len()

        # Feature Engineering - Create 20+ features
        print("\n🎯 Engineering Features...")

        # 1. Family composition features
        df['total_travelers'] = df['NumberOfAdults'] + df['NumberOfChildren']
        df['has_children'] = (df['NumberOfChildren'] > 0).astype(int)
        df['children_ratio'] = df['NumberOfChildren'] / (df['total_travelers'] + 1e-8)
        df['is_solo'] = (df['total_travelers'] == 1).astype(int)
        df['is_couple'] = (df['total_travelers'] == 2).astype(int)
        df['is_family'] = (df['total_travelers'] >= 3).astype(int)
        df['is_large_group'] = (df['total_travelers'] > 5).astype(int)

        # 2. Rating features
        df['rating_normalized'] = (df['Rating'] - df['Rating'].min()) / (df['Rating'].max() - df['Rating'].min() + 1e-8)
        df['experience_normalized'] = (df['ExperienceRating'] - df['ExperienceRating'].min()) / (df['ExperienceRating'].max() - df['ExperienceRating'].min() + 1e-8)
        df['combined_rating'] = (df['rating_normalized'] + df['experience_normalized']) / 2
        df['high_rating'] = (df['Rating'] >= 4.0).astype(int)

        # 3. Popularity features
        df['popularity_normalized'] = (df['Popularity'] - df['Popularity'].min()) / (df['Popularity'].max() - df['Popularity'].min() + 1e-8)
        df['is_popular'] = (df['Popularity'] > df['Popularity'].quantile(0.75)).astype(int)

        # 4. Time-based features
        if 'VisitDate' in df.columns:
            df['VisitDate'] = pd.to_datetime(df['VisitDate'], errors='coerce')
            df['visit_month'] = df['VisitDate'].dt.month.fillna(6)
            df['visit_quarter'] = df['VisitDate'].dt.quarter.fillna(2)
            df['visit_season'] = df['visit_month'].map({
                12: 'Winter', 1: 'Winter', 2: 'Winter',
                3: 'Spring', 4: 'Spring', 5: 'Spring',
                6: 'Summer', 7: 'Summer', 8: 'Summer',
                9: 'Fall', 10: 'Fall', 11: 'Fall'
            })

        # 5. Preference features
        df['pref_cleaned'] = df['Preferences'].str.lower().str.strip()
        df['is_adventure'] = df['pref_cleaned'].str.contains('adventure|trek|hiking', na=False).astype(int)
        df['is_cultural'] = df['pref_cleaned'].str.contains('cultural|heritage|historical', na=False).astype(int)
        df['is_beach'] = df['pref_cleaned'].str.contains('beach|coastal|sea', na=False).astype(int)
        df['is_nature'] = df['pref_cleaned'].str.contains('nature|wildlife|forest', na=False).astype(int)
        df['is_religious'] = df['pref_cleaned'].str.contains('religious|temple|spiritual', na=False).astype(int)

        # 6. Encode categorical variables
        df['user_encoded'] = self.user_encoder.fit_transform(df['UserID'].astype(str))
        df['dest_encoded'] = self.dest_encoder.fit_transform(df['DestinationID_x'].astype(str))
        df['state_encoded'] = self.state_encoder.fit_transform(df['State'].astype(str))
        df['type_encoded'] = self.type_encoder.fit_transform(df['Type'].astype(str))
        df['time_encoded'] = self.time_encoder.fit_transform(df['BestTimeToVisit'].astype(str))
        df['pref_encoded'] = self.pref_encoder.fit_transform(df['pref_cleaned'].astype(str))

        # 7. Destination aggregated features
        dest_stats = df.groupby('DestinationID_x').agg({
            'Rating': ['mean', 'std', 'count'],
            'NumberOfChildren': 'mean',
            'total_travelers': 'mean',
            'Popularity': 'mean'
        }).reset_index()
        dest_stats.columns = ['DestinationID_x', 'dest_avg_rating', 'dest_rating_std',
                              'dest_review_count', 'dest_avg_children', 'dest_avg_travelers',
                              'dest_avg_popularity']
        df = df.merge(dest_stats, on='DestinationID_x', how='left')

        # 8. User aggregated features
        user_stats = df.groupby('UserID').agg({
            'Rating': ['mean', 'count'],
            'NumberOfChildren': 'mean',
            'NumberOfAdults': 'mean'
        }).reset_index()
        user_stats.columns = ['UserID', 'user_avg_rating', 'user_review_count',
                              'user_avg_children', 'user_avg_adults']
        df = df.merge(user_stats, on='UserID', how='left')

        # 9. State-level features
        state_stats = df.groupby('State').agg({
            'Rating': 'mean',
            'Popularity': 'mean',
            'DestinationID_x': 'nunique'
        }).reset_index()
        state_stats.columns = ['State', 'state_avg_rating', 'state_avg_popularity', 'state_dest_count']
        df = df.merge(state_stats, on='State', how='left')

        self.df_processed = df

        print(f"\n✓ Preprocessed {len(df):,} records")
        print(f"\n📊 Feature Summary:")
        print(f"  Unique Users: {df['user_encoded'].nunique():,}")
        print(f"  Unique Destinations: {df['dest_encoded'].nunique():,}")
        print(f"  Unique States: {df['state_encoded'].nunique()}")
        print(f"  Unique Types: {df['type_encoded'].nunique()}")
        print(f"  Unique Preferences: {df['pref_encoded'].nunique()}")
        print(f"  Total Features: {len(df.columns)}")

        return df

    def build_advanced_model(self):
        """Build state-of-the-art hybrid deep learning model"""
        print("\n" + "=" * 70)
        print("🧠 BUILDING ADVANCED DEEP LEARNING MODEL")
        print("=" * 70)

        n_users = self.df_processed['user_encoded'].nunique()
        n_destinations = self.df_processed['dest_encoded'].nunique()
        n_states = self.df_processed['state_encoded'].nunique()
        n_types = self.df_processed['type_encoded'].nunique()
        n_times = self.df_processed['time_encoded'].nunique()
        n_prefs = self.df_processed['pref_encoded'].nunique()

        # Embedding dimensions (larger for better representation)
        user_emb_dim = 128
        dest_emb_dim = 128
        state_emb_dim = 64
        type_emb_dim = 32
        time_emb_dim = 16
        pref_emb_dim = 64

        # === USER TOWER ===
        user_input = layers.Input(shape=(1,), name='user_input')
        user_embedding = layers.Embedding(
            n_users, user_emb_dim,
            embeddings_regularizer=l2(1e-6),
            name='user_embedding'
        )(user_input)
        user_vec = layers.Flatten()(user_embedding)
        user_vec = layers.Dense(64, activation='relu')(user_vec)
        user_vec = layers.BatchNormalization()(user_vec)
        user_vec = layers.Dropout(0.3)(user_vec)

        # === DESTINATION TOWER ===
        dest_input = layers.Input(shape=(1,), name='dest_input')
        dest_embedding = layers.Embedding(
            n_destinations, dest_emb_dim,
            embeddings_regularizer=l2(1e-6),
            name='dest_embedding'
        )(dest_input)
        dest_vec = layers.Flatten()(dest_embedding)
        dest_vec = layers.Dense(64, activation='relu')(dest_vec)
        dest_vec = layers.BatchNormalization()(dest_vec)
        dest_vec = layers.Dropout(0.3)(dest_vec)

        # === CONTEXT FEATURES ===
        # State
        state_input = layers.Input(shape=(1,), name='state_input')
        state_embedding = layers.Embedding(n_states, state_emb_dim, name='state_embedding')(state_input)
        state_vec = layers.Flatten()(state_embedding)

        # Type
        type_input = layers.Input(shape=(1,), name='type_input')
        type_embedding = layers.Embedding(n_types, type_emb_dim, name='type_embedding')(type_input)
        type_vec = layers.Flatten()(type_embedding)

        # Time
        time_input = layers.Input(shape=(1,), name='time_input')
        time_embedding = layers.Embedding(n_times, time_emb_dim, name='time_embedding')(time_input)
        time_vec = layers.Flatten()(time_embedding)

        # Preferences
        pref_input = layers.Input(shape=(1,), name='pref_input')
        pref_embedding = layers.Embedding(n_prefs, pref_emb_dim, name='pref_embedding')(pref_input)
        pref_vec = layers.Flatten()(pref_embedding)

        # === NUMERICAL FEATURES ===
        numerical_input = layers.Input(shape=(15,), name='numerical_input')
        numerical_dense = layers.Dense(64, activation='relu')(numerical_input)
        numerical_dense = layers.BatchNormalization()(numerical_dense)
        numerical_dense = layers.Dropout(0.2)(numerical_dense)
        numerical_dense = layers.Dense(32, activation='relu')(numerical_dense)

        # === FEATURE FUSION ===
        # Concatenate all features
        concat = layers.Concatenate()([
            user_vec, dest_vec, state_vec, type_vec,
            time_vec, pref_vec, numerical_dense
        ])

        # === DEEP NETWORK ===
        # First block
        x = layers.Dense(512, activation='relu', kernel_regularizer=l2(1e-5))(concat)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)

        # Second block
        x = layers.Dense(256, activation='relu', kernel_regularizer=l2(1e-5))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)

        # Third block
        x = layers.Dense(128, activation='relu', kernel_regularizer=l2(1e-5))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)

        # Fourth block
        x = layers.Dense(64, activation='relu', kernel_regularizer=l2(1e-5))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)

        # === ATTENTION MECHANISM ===
        attention = layers.Dense(64, activation='tanh')(x)
        attention = layers.Dense(1, activation='softmax')(attention)
        x = layers.Multiply()([x, attention])

        # === OUTPUT LAYER ===
        output = layers.Dense(1, activation='sigmoid', name='rating_output')(x)

        # === BUILD MODEL ===
        model = Model(
            inputs=[
                user_input, dest_input, state_input,
                type_input, time_input, pref_input,
                numerical_input
            ],
            outputs=output,
            name='AdvancedTourismRecommender'
        )

        # Use fixed learning rate (ReduceLROnPlateau will handle scheduling)
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')]
        )

        self.model = model

        print("\n🏗️ Model Architecture:")
        model.summary()

        total_params = model.count_params()
        print(f"\n📊 Model Statistics:")
        print(f"  Total Parameters: {total_params:,}")
        print(f"  Trainable Parameters: {total_params:,}")

        return model

    def train_model(self, epochs=30, batch_size=128, validation_split=0.2):
        """Train the advanced recommendation model"""
        print("\n" + "=" * 70)
        print("🎓 TRAINING ADVANCED MODEL")
        print("=" * 70)

        # Prepare features
        numerical_features = [
            'NumberOfAdults', 'NumberOfChildren', 'total_travelers',
            'has_children', 'children_ratio', 'is_family',
            'Popularity', 'dest_avg_rating', 'dest_review_count',
            'dest_avg_children', 'user_avg_rating', 'user_review_count',
            'state_avg_rating', 'is_adventure', 'is_cultural'
        ]

        X_numerical = self.df_processed[numerical_features].values
        X_numerical = self.feature_scaler.fit_transform(X_numerical)

        X_train_dict = {
            'user_input': self.df_processed['user_encoded'].values,
            'dest_input': self.df_processed['dest_encoded'].values,
            'state_input': self.df_processed['state_encoded'].values,
            'type_input': self.df_processed['type_encoded'].values,
            'time_input': self.df_processed['time_encoded'].values,
            'pref_input': self.df_processed['pref_encoded'].values,
            'numerical_input': X_numerical
        }

        y = self.df_processed['rating_normalized'].values

        # Callbacks
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            verbose=1
        )

        checkpoint = ModelCheckpoint(
            'best_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )

        # Train
        print(f"\n🚀 Training Configuration:")
        print(f"  Total Samples: {len(y):,}")
        print(f"  Validation Split: {validation_split*100}%")
        print(f"  Batch Size: {batch_size}")
        print(f"  Max Epochs: {epochs}")

        history = self.model.fit(
            X_train_dict, y,
            validation_split=validation_split,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stop, reduce_lr, checkpoint],
            verbose=1
        )

        print("\n✓ Training Completed!")
        print(f"\n📊 Final Metrics:")
        print(f"  Training Loss: {history.history['loss'][-1]:.4f}")
        print(f"  Validation Loss: {history.history['val_loss'][-1]:.4f}")
        print(f"  Training RMSE: {history.history['rmse'][-1]:.4f}")
        print(f"  Validation RMSE: {history.history['val_rmse'][-1]:.4f}")

        print("\n✓ Model trained and saved as 'best_model.h5'")

        return history


# ========================================
# MAIN EXECUTION
# ========================================

def main():
    """Main execution function - Training Only"""

    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║   🌟 ADVANCED TOURISM RECOMMENDATION SYSTEM 🌟                  ║
    ║                  TRAINING MODE                                   ║
    ║                                                                  ║
    ║   Features:                                                      ║
    ║   ✓ Deep Learning with Pre-trained Embeddings                  ║
    ║   ✓ Neural Collaborative Filtering (NCF)                       ║
    ║   ✓ Attention Mechanism                                        ║
    ║   ✓ 20+ Engineered Features                                    ║
    ║   ✓ State-of-the-art Architecture                              ║
    ║                                                                  ║
    ║   Key Parameters (5-6):                                         ║
    ║   1. State - Location preference                               ║
    ║   2. Number of Adults - Group size                             ║
    ║   3. Number of Children - Family composition                   ║
    ║   4. Preferences - Interests (Adventure/Cultural/Beach/etc)    ║
    ║   5. Best Time to Visit - Seasonal preference                  ║
    ║   6. Type - Destination type (Historical/Nature/Religious)     ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)

    # Get CSV path
    csv_path = "/content/final_df (1).csv"

    # Initialize system
    recommender = AdvancedTourismRecommender(csv_path)

    # Pipeline
    print("\n" + "=" * 70)
    print("🚀 STARTING TRAINING PIPELINE")
    print("=" * 70)

    # Step 1: Load data
    recommender.load_and_analyze_data()

    # Step 2: Preprocess
    recommender.preprocess_data()

    # Step 3: Build model
    recommender.build_advanced_model()

    # Step 4: Train model (30 epochs)
    recommender.train_model(epochs=30, batch_size=128)

    print("\n" + "=" * 70)
    print("✓ TRAINING PIPELINE COMPLETED SUCCESSFULLY!")
    print("=" * 70)
    print("\n✓ Model saved as 'best_model.h5'")
    print("✓ System ready for deployment")

    return recommender


if __name__ == "__main__":
    recommender = main()

✓ TensorFlow loaded successfully

    ╔══════════════════════════════════════════════════════════════════╗
    ║   🌟 ADVANCED TOURISM RECOMMENDATION SYSTEM 🌟                  ║
    ║                  TRAINING MODE                                   ║
    ║                                                                  ║
    ║   Features:                                                      ║
    ║   ✓ Deep Learning with Pre-trained Embeddings                  ║
    ║   ✓ Neural Collaborative Filtering (NCF)                       ║
    ║   ✓ Attention Mechanism                                        ║
    ║   ✓ 20+ Engineered Features                                    ║
    ║   ✓ State-of-the-art Architecture                              ║
    ║                                                                  ║
    ║   Key Parameters (5-6):                                         ║
    ║   1. State - Location preference                               ║
    ║   2. Number of Adults - Gro

Model: "AdvancedTourismRecommender"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dest_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 128)    │    232,960 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dest_embedding      │ (None, 1, 128)    │    256,000 │ dest_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical_input     │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 128)       │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_13          │ (None, 128)       │          0 │ dest_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 64)        │      1,024 │ numerical_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 64)        │      8,256 │ flatten_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 64)        │      8,256 │ flatten_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ type_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pref_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_22[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_20[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_21[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_embedding     │ (None, 1, 64)     │      2,304 │ state_input[0][0

 Total params: 865,714 (3.30 MB)

 Trainable params: 863,410 (3.29 MB)

 Non-trainable params: 2,304 (9.00 KB)


📊 Model Statistics:
  Total Parameters: 865,714
  Trainable Parameters: 865,714

🎓 TRAINING ADVANCED MODEL

🚀 Training Configuration:
  Total Samples: 18,012
  Validation Split: 20.0%
  Batch Size: 128
  Max Epochs: 30
Epoch 1/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1523 - mae: 0.3034 - mse: 0.1417 - rmse: 0.3754
Epoch 1: val_loss improved from inf to 0.09477, saving model to best_model.h5


113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 87ms/step - loss: 0.1520 - mae: 0.3031 - mse: 0.1415 - rmse: 0.3750 - val_loss: 0.0948 - val_mae: 0.2495 - val_mse: 0.0842 - val_rmse: 0.2901 - learning_rate: 0.0010
Epoch 2/30
106/113 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0837 - mae: 0.2091 - mse: 0.0731 - rmse: 0.2703
Epoch 2: val_loss improved from 0.09477 to 0.07464, saving model to best_model.h5


113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0835 - mae: 0.2088 - mse: 0.0729 - rmse: 0.2699 - val_loss: 0.0746 - val_mae: 0.2057 - val_mse: 0.0641 - val_rmse: 0.2532 - learning_rate: 0.0010
Epoch 3/30
108/113 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0733 - mae: 0.1930 - mse: 0.0628 - rmse: 0.2505
Epoch 3: val_loss did not improve from 0.07464
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0731 - mae: 0.1928 - mse: 0.0626 - rmse: 0.2502 - val_loss: 0.0752 - val_mae: 0.2017 - val_mse: 0.0648 - val_rmse: 0.2545 - learning_rate: 0.0010
Epoch 4/30
105/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0665 - mae: 0.1825 - mse: 0.0562 - rmse: 0.2370
Epoch 4: val_loss did not improve from 0.07464
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0664 - mae: 0.1822 - mse: 0.0560 - rmse: 0.2366 - val_loss: 0.0770 - val_mae: 0.2047 - val_mse: 0.0668 - val_rmse: 0.2585 - learning_rate: 0.0010
Epoch 5/30
106/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0589 - mae: 0.1688 - mse: 0.048

In [21]:
"""
Tourism Recommendation System - TESTING CODE ONLY
Load pre-trained model and generate recommendations
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)


class TourismRecommenderTester:
    """
    Testing class for pre-trained tourism recommendation model
    """

    def __init__(self, model_path, csv_path):
        self.model_path = model_path
        self.csv_path = csv_path
        self.model = None
        self.df_processed = None
        self.destination_features = None

        # Encoders (will be fitted from data)
        self.user_encoder = LabelEncoder()
        self.dest_encoder = LabelEncoder()
        self.state_encoder = LabelEncoder()
        self.type_encoder = LabelEncoder()
        self.time_encoder = LabelEncoder()
        self.pref_encoder = LabelEncoder()
        self.feature_scaler = StandardScaler()

    def load_model(self):
        """Load pre-trained model"""
        print("=" * 70)
        print("📦 LOADING PRE-TRAINED MODEL")
        print("=" * 70)

        # Load model with custom objects
        self.model = keras.models.load_model(
            self.model_path,
            custom_objects={
                'mse': keras.losses.MeanSquaredError(),
                'mae': keras.losses.MeanAbsoluteError(),
            },
            compile=False  # Skip compilation
        )

        # Recompile with correct metrics
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')]
        )

        print(f"\n✓ Model loaded from: {self.model_path}")
        print(f"✓ Model parameters: {self.model.count_params():,}")

        return self.model

    def load_data(self):
        """Load and preprocess data for inference"""
        print("\n" + "=" * 70)
        print("📊 LOADING DATA FOR INFERENCE")
        print("=" * 70)

        df = pd.read_csv(self.csv_path)
        print(f"\n✓ Loaded {len(df):,} records")

        # Preprocess (same as training)
        df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(df['Rating'].median())
        df['ExperienceRating'] = pd.to_numeric(df['ExperienceRating'], errors='coerce').fillna(df['Rating'])
        df['NumberOfAdults'] = pd.to_numeric(df['NumberOfAdults'], errors='coerce').fillna(2)
        df['NumberOfChildren'] = pd.to_numeric(df['NumberOfChildren'], errors='coerce').fillna(0)
        df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce').fillna(df['Popularity'].median())

        df['State'] = df['State'].fillna('Unknown')
        df['Type'] = df['Type'].fillna('General')
        df['BestTimeToVisit'] = df['BestTimeToVisit'].fillna('Year-round')
        df['Preferences'] = df['Preferences'].fillna('General Tourism')

        # Feature engineering
        df['total_travelers'] = df['NumberOfAdults'] + df['NumberOfChildren']
        df['has_children'] = (df['NumberOfChildren'] > 0).astype(int)
        df['children_ratio'] = df['NumberOfChildren'] / (df['total_travelers'] + 1e-8)
        df['is_family'] = (df['total_travelers'] >= 3).astype(int)

        df['rating_normalized'] = (df['Rating'] - df['Rating'].min()) / (df['Rating'].max() - df['Rating'].min() + 1e-8)
        df['experience_normalized'] = (df['ExperienceRating'] - df['ExperienceRating'].min()) / (df['ExperienceRating'].max() - df['ExperienceRating'].min() + 1e-8)

        df['pref_cleaned'] = df['Preferences'].str.lower().str.strip()
        df['is_adventure'] = df['pref_cleaned'].str.contains('adventure|trek|hiking', na=False).astype(int)
        df['is_cultural'] = df['pref_cleaned'].str.contains('cultural|heritage|historical', na=False).astype(int)

        # Encode
        df['user_encoded'] = self.user_encoder.fit_transform(df['UserID'].astype(str))
        df['dest_encoded'] = self.dest_encoder.fit_transform(df['DestinationID_x'].astype(str))
        df['state_encoded'] = self.state_encoder.fit_transform(df['State'].astype(str))
        df['type_encoded'] = self.type_encoder.fit_transform(df['Type'].astype(str))
        df['time_encoded'] = self.time_encoder.fit_transform(df['BestTimeToVisit'].astype(str))
        df['pref_encoded'] = self.pref_encoder.fit_transform(df['pref_cleaned'].astype(str))

        # Aggregated features
        dest_stats = df.groupby('DestinationID_x').agg({
            'Rating': ['mean', 'count'],
            'NumberOfChildren': 'mean',
            'Popularity': 'mean'
        }).reset_index()
        dest_stats.columns = ['DestinationID_x', 'dest_avg_rating', 'dest_review_count',
                              'dest_avg_children', 'dest_avg_popularity']
        df = df.merge(dest_stats, on='DestinationID_x', how='left')

        user_stats = df.groupby('UserID').agg({
            'Rating': ['mean', 'count'],
        }).reset_index()
        user_stats.columns = ['UserID', 'user_avg_rating', 'user_review_count']
        df = df.merge(user_stats, on='UserID', how='left')

        state_stats = df.groupby('State').agg({
            'Rating': 'mean',
        }).reset_index()
        state_stats.columns = ['State', 'state_avg_rating']
        df = df.merge(state_stats, on='State', how='left')

        self.df_processed = df

        # Fit scaler
        numerical_features = [
            'NumberOfAdults', 'NumberOfChildren', 'total_travelers',
            'has_children', 'children_ratio', 'is_family',
            'Popularity', 'dest_avg_rating', 'dest_review_count',
            'dest_avg_children', 'user_avg_rating', 'user_review_count',
            'state_avg_rating', 'is_adventure', 'is_cultural'
        ]
        X_numerical = df[numerical_features].values
        self.feature_scaler.fit(X_numerical)

        # Build destination cache
        self._build_destination_cache()

        print(f"\n✓ Data preprocessed successfully")
        print(f"  Unique States: {df['State'].nunique()}")
        print(f"  Unique Destinations: {df['DestinationID_x'].nunique()}")

        return df

    def _build_destination_cache(self):
        """Cache destination features"""
        dest_features = self.df_processed.groupby('DestinationID_x').agg({
            'Name_x': 'first',
            'State': 'first',
            'Type': 'first',
            'BestTimeToVisit': 'first',
            'dest_avg_rating': 'first',
            'dest_review_count': 'first',
            'Popularity': 'first',
            'dest_encoded': 'first',
            'state_encoded': 'first',
            'type_encoded': 'first',
            'time_encoded': 'first',
            'dest_avg_children': 'first'
        }).reset_index()

        self.destination_features = dest_features
        print(f"\n✓ Cached {len(dest_features)} destinations")

    def get_recommendations(self, state, num_adults, num_children,
                           preferences='General Tourism',
                           best_time='Year-round',
                           destination_type=None,
                           top_n=10):
        """
        Generate recommendations using pre-trained model

        Args:
            state: State name (e.g., 'Kerala', 'Tamil Nadu')
            num_adults: Number of adults (1-10)
            num_children: Number of children (0-5)
            preferences: User interests (e.g., 'Adventure', 'Cultural', 'Beach')
            best_time: Best time to visit (e.g., 'Winter', 'Summer', 'Year-round')
            destination_type: Type filter (optional)
            top_n: Number of recommendations
        """
        print("\n" + "=" * 70)
        print("🎯 GENERATING RECOMMENDATIONS")
        print("=" * 70)

        print(f"\n👤 Input Parameters:")
        print(f"  📍 State: {state}")
        print(f"  👨 Adults: {num_adults}")
        print(f"  👶 Children: {num_children}")
        print(f"  ❤️  Preferences: {preferences}")
        print(f"  📅 Best Time: {best_time}")
        if destination_type:
            print(f"  🏛️  Type: {destination_type}")

        # Filter by state
        candidates = self.destination_features[
            self.destination_features['State'].str.lower() == state.lower()
        ].copy()

        if len(candidates) == 0:
            print(f"\n❌ No destinations found for: {state}")
            print(f"\n📍 Available states:")
            available = sorted(self.destination_features['State'].unique())
            for i in range(0, len(available), 5):
                print(f"  {', '.join(available[i:i+5])}")
            return pd.DataFrame()

        # Filter by type if specified
        if destination_type:
            type_candidates = candidates[
                candidates['Type'].str.lower().str.contains(destination_type.lower(), na=False)
            ]
            if len(type_candidates) > 0:
                candidates = type_candidates

        print(f"\n✓ Found {len(candidates)} candidates in {state}")

        # Prepare user features
        total_travelers = num_adults + num_children
        has_children = 1 if num_children > 0 else 0
        children_ratio = num_children / (total_travelers + 1e-8)
        is_family = 1 if total_travelers >= 3 else 0

        # Encode preferences
        pref_cleaned = preferences.lower().strip()
        if pref_cleaned not in self.pref_encoder.classes_:
            pref_encoded = 0
        else:
            pref_encoded = self.pref_encoder.transform([pref_cleaned])[0]

        # Encode time
        if best_time not in self.time_encoder.classes_:
            time_encoded = 0
        else:
            time_encoded = self.time_encoder.transform([best_time])[0]

        # Find proxy user
        similar_mask = (
            (self.df_processed['has_children'] == has_children) &
            (self.df_processed['total_travelers'].between(total_travelers-1, total_travelers+1))
        )
        similar_users = self.df_processed[similar_mask]['user_encoded'].unique()

        if len(similar_users) > 0:
            proxy_user = int(np.median(similar_users))
        else:
            proxy_user = int(self.df_processed['user_encoded'].median())

        # Prepare numerical features
        n_candidates = len(candidates)

        avg_dest_rating = candidates['dest_avg_rating'].mean()
        avg_dest_count = candidates['dest_review_count'].mean()
        avg_popularity = candidates['Popularity'].mean()

        if len(similar_users) > 0:
            user_avg_rating = self.df_processed[
                self.df_processed['user_encoded'].isin(similar_users)
            ]['Rating'].mean()
            user_avg_count = self.df_processed[
                self.df_processed['user_encoded'].isin(similar_users)
            ].groupby('user_encoded').size().mean()
        else:
            user_avg_rating = self.df_processed['Rating'].mean()
            user_avg_count = 5

        state_avg_rating = self.df_processed[
            self.df_processed['State'].str.lower() == state.lower()
        ]['Rating'].mean()

        is_adventure = 1 if 'adventure' in pref_cleaned or 'trek' in pref_cleaned else 0
        is_cultural = 1 if 'cultural' in pref_cleaned or 'heritage' in pref_cleaned else 0

        numerical_features = np.array([
            num_adults, num_children, total_travelers,
            has_children, children_ratio, is_family,
            avg_popularity, avg_dest_rating, avg_dest_count,
            num_children, user_avg_rating, user_avg_count,
            state_avg_rating, is_adventure, is_cultural
        ]).reshape(1, -1)

        numerical_scaled = self.feature_scaler.transform(numerical_features)
        numerical_scaled = np.repeat(numerical_scaled, n_candidates, axis=0)

        # Prepare model inputs
        X_pred = {
            'user_input': np.full(n_candidates, proxy_user),
            'dest_input': candidates['dest_encoded'].values,
            'state_input': candidates['state_encoded'].values,
            'type_input': candidates['type_encoded'].values,
            'time_input': np.full(n_candidates, time_encoded),
            'pref_input': np.full(n_candidates, pref_encoded),
            'numerical_input': numerical_scaled
        }

        # Predict
        print(f"\n🔮 Running AI predictions...")
        predictions = self.model.predict(X_pred, verbose=0, batch_size=256).flatten()

        # Calculate final scores
        candidates = candidates.copy()
        candidates['predicted_score'] = predictions

        candidates['popularity_score'] = (
            candidates['predicted_score'] * 0.40 +
            candidates['dest_avg_rating'].fillna(0.5) * 0.30 +
            np.log1p(candidates['dest_review_count'].fillna(1)) * 0.15 +
            (candidates['Popularity'].fillna(50) / 100) * 0.15
        )

        # Apply boosts
        if num_children > 0:
            family_boost = candidates['dest_avg_children'].fillna(0) > 0.5
            candidates.loc[family_boost, 'popularity_score'] *= 1.2

        time_match = candidates['BestTimeToVisit'].str.lower().str.contains(
            best_time.lower(), na=False
        )
        candidates.loc[time_match, 'popularity_score'] *= 1.1

        # Get top N
        recommendations = candidates.nlargest(top_n, 'popularity_score')

        # Format output
        output = recommendations[[
            'Name_x', 'State', 'Type', 'BestTimeToVisit',
            'predicted_score', 'dest_avg_rating', 'dest_review_count',
            'Popularity', 'popularity_score'
        ]].copy()

        output.columns = [
            'Destination', 'State', 'Type', 'Best_Time',
            'AI_Score', 'Avg_Rating', 'Reviews', 'Popularity', 'Final_Score'
        ]

        output['AI_Score'] = output['AI_Score'].round(3)
        output['Avg_Rating'] = output['Avg_Rating'].round(2)
        output['Final_Score'] = output['Final_Score'].round(3)

        print(f"\n✨ TOP {top_n} RECOMMENDATIONS:\n")
        print(output.to_string(index=False))

        return output

    def show_available_options(self):
        """Display available options for user"""
        print("\n" + "=" * 70)
        print("📋 AVAILABLE OPTIONS")
        print("=" * 70)

        print(f"\n📍 States ({self.df_processed['State'].nunique()}):")
        states = sorted(self.df_processed['State'].unique())
        for i in range(0, len(states), 5):
            print(f"  {', '.join(states[i:i+5])}")

        print(f"\n🏛️  Types ({self.df_processed['Type'].nunique()}):")
        types = sorted(self.df_processed['Type'].unique())
        for i in range(0, len(types), 4):
            print(f"  {', '.join(types[i:i+4])}")

        print(f"\n📅 Best Times:")
        times = sorted(self.df_processed['BestTimeToVisit'].unique()[:10])
        print(f"  {', '.join(times)}")

        print(f"\n❤️  Preference Examples:")
        print(f"  Adventure, Cultural, Beach, Nature, Religious, Historical")


# ========================================
# TESTING FUNCTIONS
# ========================================

def test_single_recommendation(tester):
    """Test with a single query"""
    print("\n" + "=" * 70)
    print("🧪 TEST 1: SINGLE RECOMMENDATION")
    print("=" * 70)

    result = tester.get_recommendations(
        state='Kerala',
        num_adults=2,
        num_children=1,
        preferences='Beach',
        best_time='Winter',
        top_n=5
    )

    return result


def test_multiple_queries(tester):
    """Test with multiple different queries"""
    print("\n" + "=" * 70)
    print("🧪 TEST 2: MULTIPLE QUERIES")
    print("=" * 70)

    test_cases = [
        {'state': 'Tamil Nadu', 'adults': 2, 'children': 0, 'pref': 'Cultural', 'time': 'Year-round'},
        {'state': 'Rajasthan', 'adults': 3, 'children': 2, 'pref': 'Historical', 'time': 'Winter'},
        {'state': 'Goa', 'adults': 2, 'children': 0, 'pref': 'Beach', 'time': 'Winter'},
        {'state': 'Karnataka', 'adults': 4, 'children': 1, 'pref': 'Nature', 'time': 'Summer'},
    ]

    results = []
    for i, test in enumerate(test_cases, 1):
        print(f"\n--- Query {i} ---")
        result = tester.get_recommendations(
            state=test['state'],
            num_adults=test['adults'],
            num_children=test['children'],
            preferences=test['pref'],
            best_time=test['time'],
            top_n=3
        )
        results.append(result)

    return results


def interactive_test(tester):
    """Interactive testing mode"""
    print("\n" + "=" * 70)
    print("🎮 INTERACTIVE TESTING MODE")
    print("=" * 70)

    tester.show_available_options()

    print(f"\n" + "-" * 70)
    state = input("📍 Enter state: ")
    adults = int(input("👨 Number of adults: "))
    children = int(input("👶 Number of children: "))
    preferences = input("❤️  Preferences: ") or "General Tourism"
    best_time = input("📅 Best time: ") or "Year-round"
    dest_type = input("🏛️  Type (optional): ") or None
    top_n = int(input("🔢 Top N (default 10): ") or 10)

    return tester.get_recommendations(
        state, adults, children, preferences, best_time, dest_type, top_n
    )


# ========================================
# MAIN EXECUTION - USER INPUT BASED
# ========================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║   🧪 TOURISM RECOMMENDER - TESTING MODE                        ║
    ║                                                                  ║
    ║   Load pre-trained model and generate recommendations          ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)

    # Configuration
    MODEL_PATH = "best_model.h5"
    CSV_PATH = "/content/final_df (1).csv"

    # Initialize tester
    print("\n🔄 Initializing system...")
    tester = TourismRecommenderTester(MODEL_PATH, CSV_PATH)

    # Load model and data
    tester.load_model()
    tester.load_data()

    # Show available options
    tester.show_available_options()

    # Interactive recommendation loop
    print("\n" + "=" * 70)
    print("🎮 INTERACTIVE RECOMMENDATION SYSTEM")
    print("=" * 70)

    while True:
        print("\n" + "-" * 70)
        print("📝 Enter your travel preferences:")
        print("-" * 70)

        # Get user inputs
        state = input("\n📍 Enter state (e.g., Kerala, Tamil Nadu, Rajasthan): ").strip()

        if not state:
            print("❌ State is required!")
            continue

        try:
            num_adults = int(input("👨 Number of adults (1-10): "))
            num_children = int(input("👶 Number of children (0-5): "))

            if num_adults < 1 or num_adults > 10:
                print("❌ Number of adults must be between 1 and 10")
                continue

            if num_children < 0 or num_children > 5:
                print("❌ Number of children must be between 0 and 5")
                continue

        except ValueError:
            print("❌ Please enter valid numbers!")
            continue

        preferences = input("❤️  Your preferences (e.g., Adventure, Beach, Cultural) [Press Enter for 'General Tourism']: ").strip()
        if not preferences:
            preferences = "General Tourism"

        best_time = input("📅 Best time to visit (e.g., Winter, Summer, Year-round) [Press Enter for 'Year-round']: ").strip()
        if not best_time:
            best_time = "Year-round"

        destination_type = input("🏛️  Destination type (e.g., Historical, Nature, Religious) [Press Enter to skip]: ").strip()
        if not destination_type:
            destination_type = None

        try:
            top_n = input("🔢 Number of recommendations (default 10): ").strip()
            top_n = int(top_n) if top_n else 10

            if top_n < 1 or top_n > 50:
                print("⚠️  Setting recommendations to 10 (valid range: 1-50)")
                top_n = 10
        except ValueError:
            print("⚠️  Invalid number, using default 10")
            top_n = 10

        # Generate recommendations
        try:
            result = tester.get_recommendations(
                state=state,
                num_adults=num_adults,
                num_children=num_children,
                preferences=preferences,
                best_time=best_time,
                destination_type=destination_type,
                top_n=top_n
            )

            if len(result) > 0:
                print("\n✅ Recommendations generated successfully!")

        except Exception as e:
            print(f"\n❌ Error generating recommendations: {str(e)}")

        # Ask to continue
        print("\n" + "-" * 70)
        cont = input("🔄 Would you like to get more recommendations? (yes/no): ").strip().lower()

        if cont not in ['yes', 'y']:
            break

    print("\n" + "=" * 70)
    print("✅ TESTING COMPLETED! Thank you for using Tourism Recommender!")
    print("=" * 70)


    ╔══════════════════════════════════════════════════════════════════╗
    ║   🧪 TOURISM RECOMMENDER - TESTING MODE                        ║
    ║                                                                  ║
    ║   Load pre-trained model and generate recommendations          ║
    ╚══════════════════════════════════════════════════════════════════╝
    

🔄 Initializing system...
📦 LOADING PRE-TRAINED MODEL

✓ Model loaded from: best_model.h5
✓ Model parameters: 865,714

📊 LOADING DATA FOR INFERENCE

✓ Loaded 18,012 records

✓ Cached 2000 destinations

✓ Data preprocessed successfully
  Unique States: 36
  Unique Destinations: 2000

📋 AVAILABLE OPTIONS

📍 States (36):
  Andaman & Nicobar, Andhra Pradesh, Arunachal Pradesh, Assam, Bihar
  Chandigarh, Chhattisgarh, Daman & Diu, Delhi, Goa
  Gujarat, Haryana, Himachal Pradesh, Jammu & Kashmir, Jharkhand
  Karnataka, Kerala, Ladakh, Lakshadweep, Madhya Pradesh
  Maharashtra, Manipur, Meghalaya, Mizoram, Nagaland
  Odisha, Puducher

KeyboardInterrupt: Interrupted by user